In [1]:
import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_text as text

import pandas as pd

2024-02-12 16:42:24.241722: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-02-12 16:42:24.302758: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-02-12 16:42:24.302790: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-02-12 16:42:24.305562: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-02-12 16:42:24.315364: I tensorflow/core/platform/cpu_feature_guar

In [2]:
df = pd.read_csv("emobank.csv", index_col="id")
df = df.dropna()
df.head(10)

,split,V,A,D,text
id,,,,,
110CYL068_1036_1079,train,3.00,3.00,3.20,"Remember what she said in my last letter? """
110CYL068_1079_1110,test,2.80,3.10,2.80,If I wasn't working here.
110CYL068_1127_1130,train,3.00,3.00,3.00,".."""
110CYL068_1137_1188,train,3.44,3.00,3.22,Goodwill helps people get off of public assist...
110CYL068_1189_1328,train,3.55,3.27,3.46,Sherry learned through our Future Works class ...
110CYL068_1475_1558,train,3.60,3.30,3.80,Coming to Goodwill was the first step toward m...
110CYL068_1559_1601,train,3.00,3.00,3.10,"I am now... totally off of welfare."""
110CYL068_1608_1658,train,3.10,3.10,3.10,Goodwill prepares people for life-long employm...
110CYL068_163_275,train,3.25,2.88,3.00,Here's another story of success from what migh...


In [3]:
bert_preprocess = hub.KerasLayer(
    "https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3"
)
bert_encoder = hub.KerasLayer(
    "https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/3"
)

2024-02-12 16:42:25.948288: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-02-12 16:42:25.991984: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-02-12 16:42:25.992243: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-

In [4]:
train_data = df[(df["split"] == "train")]
dev_data = df[df["split"] == "dev"]
test_data = df[df["split"] == "test"]

In [5]:
X_train = train_data["text"]
y_train = train_data[["V", "A", "D"]]
X_dev = dev_data["text"]
y_dev = dev_data[["V", "A", "D"]]
X_test = test_data["text"]
y_test = test_data[["V", "A", "D"]]

In [6]:
text_input = tf.keras.layers.Input(shape=(), dtype=tf.string, name="text")
preprocessed_text = bert_preprocess(text_input)
outputs = bert_encoder(preprocessed_text)
l = tf.keras.layers.Dropout(0.1, name="dropout")(outputs["pooled_output"])
output = tf.keras.layers.Dense(3, activation="linear", name="output")(l)
model = tf.keras.Model(inputs=[text_input], outputs=output)

In [7]:
early_stopping = tf.keras.callbacks.EarlyStopping(monitor="val_loss", patience=3)

In [8]:
model.compile(optimizer="adam", loss="mean_squared_error")

In [9]:
history = model.fit(
    X_train,
    y_train,
    validation_data=(X_dev, y_dev),
    epochs=10,
    batch_size=32,
    callbacks=[early_stopping],
)

Epoch 1/10


2024-02-12 16:42:34.308521: I external/local_tsl/tsl/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory


  1/252 [..............................] - ETA: 20:40 - loss: 15.7526

2024-02-12 16:42:35.953333: I external/local_xla/xla/service/service.cc:168] XLA service 0x7f8e06657be0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2024-02-12 16:42:35.953356: I external/local_xla/xla/service/service.cc:176]   StreamExecutor device (0): NVIDIA GeForce RTX 3060 Laptop GPU, Compute Capability 8.6
2024-02-12 16:42:35.956714: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:269] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2024-02-12 16:42:35.969259: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:454] Loaded cuDNN version 8904
I0000 00:00:1707736356.029438   24249 device_compiler.h:186] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


252/252 [==============================] - 180s 698ms/step - loss: 0.6154 - val_loss: 0.1153
Epoch 2/10
252/252 [==============================] - 175s 695ms/step - loss: 0.1831 - val_loss: 0.0868
Epoch 3/10
252/252 [==============================] - 171s 679ms/step - loss: 0.1588 - val_loss: 0.0845
Epoch 4/10
252/252 [==============================] - 183s 726ms/step - loss: 0.1433 - val_loss: 0.0729
Epoch 5/10
252/252 [==============================] - 191s 758ms/step - loss: 0.1297 - val_loss: 0.0719
Epoch 6/10
252/252 [==============================] - 191s 760ms/step - loss: 0.1204 - val_loss: 0.0777
Epoch 7/10
252/252 [==============================] - 182s 722ms/step - loss: 0.1097 - val_loss: 0.0679
Epoch 8/10
252/252 [==============================] - 173s 685ms/step - loss: 0.1022 - val_loss: 0.0641
Epoch 9/10
252/252 [==============================] - 168s 666ms/step - loss: 0.0943 - val_loss: 0.0678
Epoch 10/10
 45/252 [====>.........................] - ETA: 2:01 - loss: 0.

In [ ]:
history_dict = history.history


loss = history_dict["loss"]
val_loss = history_dict["val_loss"]


print(f"\nloss = {loss}")
print(f"\nvalue loss = {val_loss}")


loss = [0.6055861711502075, 0.301457941532135, 0.2586796283721924, 0.2359689176082611, 0.22238411009311676, 0.20686794817447662, 0.20147627592086792, 0.1894906610250473, 0.18470416963100433]

value loss = [0.1069352999329567, 0.08140023052692413, 0.07504323869943619, 0.08290749788284302, 0.07285863906145096, 0.07257340103387833, 0.07348567992448807, 0.07373587042093277, 0.07576337456703186]


In [ ]:
# Predict the values for the test set
y_pred = model.predict(X_test)

# Calculate the metrics

32/32 [==============================] - 8s 237ms/step


In [ ]:
from sklearn.metrics import r2_score

In [ ]:
y_test_array = y_test.to_numpy().flatten()
y_pred_array = y_pred.flatten()

# Calculate the metrics
mse = tf.keras.losses.mean_squared_error(y_test_array, y_pred_array)
rmse = tf.sqrt(mse)
mae = tf.keras.losses.mean_absolute_error(y_test_array, y_pred_array)
r2 = r2_score(y_test_array, y_pred_array)

print(f"Mean Squared Error (MSE): {mse.numpy()}")
print(f"Root Mean Squared Error (RMSE): {rmse.numpy()}")
print(f"Mean Absolute Error (MAE): {mae.numpy()}")
print(f"R-squared (R²): {r2}")

Mean Squared Error (MSE): 0.07853340357542038
Root Mean Squared Error (RMSE): 0.2802381217479706
Mean Absolute Error (MAE): 0.21462373435497284
R-squared (R²): -0.03501497006868548


In [ ]:
import numpy as np

252/252 [==============================] - 56s 224ms/step


In [ ]:
model.save("text_emotion")

INFO:tensorflow:Assets written to: text_emotion/assets


INFO:tensorflow:Assets written to: text_emotion/assets


In [ ]:
print(X_train[0])

Remember what she said in my last letter? "


/tmp/ipykernel_4488/39969900.py:1: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  print(X_train[0])


In [ ]:
prediction = model.predict(["Remember what she said in my last letter? "])
V = prediction[0][0] / 5 * 100
A = prediction[0][1] / 5 * 100
D = prediction[0][2] / 5 * 100
print(f"Valance = {V} %\nArusal = {A} %\nDominance = {D} %")

1/1 [==============================] - 0s 31ms/step
Valance = 58.06051254272461 %
Arusal = 59.74236011505128 %
Dominance = 59.27917480468749 %
